In [57]:
class State(object):
    def __init__(self, label, rules, dot_idx, start_idx, end_idx, idx, made_from, producer):
        self.label = label
        self.rules = rules
        self.dot_idx = dot_idx
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.idx = idx
        self.made_from = made_from
        self.producer = producer

    def next(self):
        """Returns the tag after the dot"""
        return self.rules[self.dot_idx]

    def complete(self):
        return len(self.rules) == self.dot_idx

    def __eq__(self, other):
        return (self.label == other.label and
                self.rules == other.rules and
                self.dot_idx == other.dot_idx and
                self.start_idx == other.start_idx and
                self.end_idx == other.end_idx)

    def __str__(self):
        rule_string = ''
        for i, rule in enumerate(self.rules):
            if i == self.dot_idx:
                rule_string += '· '
            rule_string += rule + ' '
        if self.dot_idx == len(self.rules):
            rule_string += '·'
        return 'S%d %s -> %s [%d,%d] %s %s' % (self.idx, self.label, rule_string, self.start_idx,
                                                self.end_idx, self.made_from, self.producer)

class Earley:
    def __init__(self, words, grammar, terminals):
        self.chart = [[] for _ in range(len(words) + 1)]
        self.current_id = 0
        self.words = words
        self.grammar = grammar
        self.terminals = terminals

    def get_new_id(self):
        self.current_id += 1
        return self.current_id - 1

    def is_terminal(self, tag):
        return tag in self.terminals

    def is_complete(self, state):
        return len(state.rules) == state.dot_idx

    def enqueue(self, state, chart_entry):
        if state not in self.chart[chart_entry]:
            self.chart[chart_entry].append(state)
        else:
            self.current_id -= 1

    def predictor(self, state):
        for production in self.grammar[state.next()]:
            self.enqueue(State(state.next(), production, 0, state.end_idx, state.end_idx, self.get_new_id(), [], 'Predictor'), state.end_idx)

    def scanner(self, state):
        if self.words[state.end_idx] in self.grammar[state.next()]:
            self.enqueue(State(state.next(), [self.words[state.end_idx]], 1, state.end_idx, state.end_idx + 1, self.get_new_id(), [], 'Scanner'), state.end_idx + 1)

    def completer(self, state):
        for s in self.chart[state.start_idx]:
            if not s.complete() and s.next() == state.label and s.end_idx == state.start_idx and s.label != '?':
                self.enqueue(State(s.label, s.rules, s.dot_idx + 1, s.start_idx, state.end_idx, self.get_new_id(), s.made_from + [state.idx], 'Completer'), state.end_idx)

    def parse(self):
        self.enqueue(State('?', ['S'], 0, 0, 0, self.get_new_id(), [], 'Dummy Start State'), 0)

        for i in range(len(self.words) + 1):
            for state in self.chart[i]:
                if not state.complete() and not self.is_terminal(state.next()):
                    if state.idx == 20: continue
                    self.predictor(state)
                elif i != len(self.words) and not state.complete() and self.is_terminal(state.next()):
                    self.scanner(state)
                else:
                    self.completer(state)

    def __str__(self):
        res = ''

        for i, chart in enumerate(self.chart):
            res += '\nChart[%d]\n' % i
            for state in chart:
                res += str(state) + '\n'

        return res

In [58]:
# S ->  NP VP | NP | PP | VP
# NP -> DT N | DT JJ N |  N
# VP -> V | NP V  | V NP | V PP
# PP -> P NP | P VP
# DT -> 'a' | 'this'
# N -> 'flight' | 'Nimal' | 'Colombo' | 'call' | 'plane' | 'Kamal' | 'Amal'
# V -> 'made' | 'wanted' | 'leaving' | 'called' | 'call'
# P -> 'from' | 'on' | 'to'
# JJ -> 'morning' | 'jet'

def example_grammmer_1():
    grammar = {
        'S':           [['NP', 'VP'], ['NP'], ['PP'], ['VP']],
        'NP':          [['DT', 'N'], ['DT', 'JJ', 'N'], ['N']],
        'VP':          [['V'], ['NP', 'V'], ['V', 'NP'], ['V', 'PP']],
        'PP':          [['P', 'NP'], ['P', 'VP']],
        'DT':          ['a', 'this' ],
        'N':           ['flight', 'Nimal', 'Colombo', 'call', 'plane', 'Kamal', 'Amal'],
        'V':           ['made', 'wanted', 'leaving', 'called', 'call'],
        'P':           ['from', 'on', 'to'],
        'JJ':          ['morning', 'jet']
    }
    terminals = ['DT', 'N', 'V', 'P', 'JJ']

    earley = Earley('Amal made a call'.split(), grammar, terminals)
    earley.parse()
    print(earley)

if __name__ == '__main__':
    example_grammmer_1()


Chart[0]
S0 ? -> · S  [0,0] [] Dummy Start State
S1 S -> · NP VP  [0,0] [] Predictor
S2 S -> · NP  [0,0] [] Predictor
S3 S -> · PP  [0,0] [] Predictor
S4 S -> · VP  [0,0] [] Predictor
S5 NP -> · DT N  [0,0] [] Predictor
S6 NP -> · DT JJ N  [0,0] [] Predictor
S7 NP -> · N  [0,0] [] Predictor
S8 PP -> · P NP  [0,0] [] Predictor
S9 PP -> · P VP  [0,0] [] Predictor
S10 VP -> · V  [0,0] [] Predictor
S11 VP -> · NP V  [0,0] [] Predictor
S12 VP -> · V NP  [0,0] [] Predictor
S13 VP -> · V PP  [0,0] [] Predictor

Chart[1]
S14 N -> Amal · [0,1] [] Scanner
S15 NP -> N · [0,1] [14] Completer
S16 S -> NP · VP  [0,1] [15] Completer
S17 S -> NP · [0,1] [15] Completer
S18 VP -> NP · V  [0,1] [15] Completer
S19 VP -> · V  [1,1] [] Predictor
S20 VP -> · NP V  [1,1] [] Predictor
S21 VP -> · V NP  [1,1] [] Predictor
S22 VP -> · V PP  [1,1] [] Predictor

Chart[2]
S23 V -> made · [1,2] [] Scanner
S24 VP -> NP V · [0,2] [15, 23] Completer
S25 VP -> V · [1,2] [23] Completer
S26 VP -> V · NP  [1,2] [23] Compl

In [35]:
def example_grammmer_2():
    grammar = {
        'S':           [['NP', 'VP'], ['Aux', 'NP', 'VP'], ['VP']],
        'NP':          [['Pronoun'], ['Proper-Noun'], ['Det', 'Nominal']],
        'Nominal':     [['Noun'], ['Nominal', 'Noun'], ['Nominal', 'PP']],
        'VP':          [['Verb'], ['Verb', 'NP'], ['Verb', 'NP', 'PP'], ['Verb', 'PP'], ['VP', 'PP']],
        'PP':          [['Prep', 'NP']],
        'Det':         ['that', 'this', 'a'],
        'Noun':        ['book', 'flight', 'meal', 'money'],
        'Verb':        ['book', 'include', 'prever'],
        'Aux':         ['does'],
        'Prep':        ['from', 'to', 'on'],
        'Proper-Noun': ['Houston', 'TWA'],
        'Pronoun':     ['he', 'she', 'it']
    }

    terminals = ['Noun', 'Verb', 'Aux', 'Prep','Pronoun', 'Proper-Noun', 'Det']

    earley = Earley(['book', 'that', 'flight'], grammar, terminals)
    earley.parse()
    print (earley)

if __name__ == '__main__':
    example_grammmer_2()


Chart[0]
S0 ? -> · S  [0,0] [] Dummy Start State
S1 S -> · NP VP  [0,0] [] Predictor
S2 S -> · Aux NP VP  [0,0] [] Predictor
S3 S -> · VP  [0,0] [] Predictor
S4 NP -> · Pronoun  [0,0] [] Predictor
S5 NP -> · Proper-Noun  [0,0] [] Predictor
S6 NP -> · Det Nominal  [0,0] [] Predictor
S7 VP -> · Verb  [0,0] [] Predictor
S8 VP -> · Verb NP  [0,0] [] Predictor
S9 VP -> · Verb NP PP  [0,0] [] Predictor
S10 VP -> · Verb PP  [0,0] [] Predictor
S11 VP -> · VP PP  [0,0] [] Predictor

Chart[1]
S12 Verb -> book · [0,1] [] Scanner
S13 VP -> Verb · [0,1] [12] Completer
S14 VP -> Verb · NP  [0,1] [12] Completer
S15 VP -> Verb · NP PP  [0,1] [12] Completer
S16 VP -> Verb · PP  [0,1] [12] Completer
S17 S -> VP · [0,1] [13] Completer
S18 VP -> VP · PP  [0,1] [13] Completer
S19 NP -> · Pronoun  [1,1] [] Predictor
S20 NP -> · Proper-Noun  [1,1] [] Predictor
S21 NP -> · Det Nominal  [1,1] [] Predictor
S22 PP -> · Prep NP  [1,1] [] Predictor
S23 ? -> S · [0,1] [17] Completer

Chart[2]
S24 Det -> that · [1,